### Project Description

1) Users who engage with both ideological groups are uncommon and receive more negative engagement compared to users who don't

2) Over time, approaching the election, users’ tweets become more negatively charged

3) Tweets (non-bridging) identified as left leaning use more negative terms while tweets identified with right ideologies use more positive terms

In [17]:
import pandas as pd
import matplotlib.pyplot as plt

# replace with your own relative file path
df = pd.read_csv("data_twitter/clean_tweets_directed.csv", sep = ",")


In [18]:
# IDENTIFYING bridging users
df['PartyName'] = df['PartyName'].str.title()
df['To-Ideology'] = df['PartyName'].map({
    'Democrats': 'left',
    'Republicans': 'right',
    'Bothparty': 'center',
    'Neither' : 'other',
})

df_filtered = df[df['To-Ideology'].isin(['left','right'])]

interactions = df_filtered.pivot_table(index='From-User-Id',
               columns='To-Ideology',
               values='Score',   
               aggfunc='count',
               fill_value=0)

len(interactions)

1284852

In [19]:
bridging_users = interactions[(interactions['left'] > 0) & (interactions['right'] > 0)].index.tolist()
df['Is_Bridging_User'] = df['From-User-Id'].isin(bridging_users)
df['Is_Recipient_Bridging'] = df['To-User-Id'].isin(bridging_users)
print(len(bridging_users))

204455


#### Consistency Checks

In [20]:
len(df[df['Is_Bridging_User']]['From-User-Id'].unique()) == len(bridging_users)

True

In [21]:
bridging_from_df = set(df[df['Is_Bridging_User']]['From-User-Id'].unique())

bridging_from_logic = set(bridging_users)

assert bridging_from_df == bridging_from_logic, "Mismatch in bridging user labeling"

In [22]:
interactions['left > 0'] = interactions['left'] > 0
interactions['right > 0'] = interactions['right'] > 0

assert all(interactions.loc[bridging_users, ['left > 0', 'right > 0']].all(axis=1)), \
       "Some bridging users do not interact with both sides"


In [23]:
assert df['To-Ideology'].notna().all(), "Missing To-Ideology values"